In [6]:
import re
import string

import pandas as pd
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [71]:
# функции для очистки текста
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_HTML(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_numbers(text):
    num = re.compile(r'\d+')
    return num.sub(r'', text)

def remove_spaces(text):
    spaces = re.compile(r'  ')
    return spaces.sub(r' ', text)

def remove_punct(text):
    punct = re.compile(r'[^\w\s]')
    return punct.sub(r'', text) 

# def remove_TASS(text):
#     tass = re.compile(r'ТАСС')
#    return tass.sub(r'', text)

def remove_months(text):
    months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь',
              'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']
    for elm in months:
        text = text.replace(elm, '')
    return text

def remove_wordsNumbers(text):
    numbers = ['год', 'млн', 'тыс', 'км', 'млрд', 'тонна', 'возраст', 'гг', 'назад']
    for i in numbers:
        text = text.replace(i, '')
    return text
    
def remove_useless(text):
    useless = ['это', 'который', 'ТАСС', 'twitter', 'понедельник', 'вторник', 'среда',
               'четверг', 'пятница', 'суббота', 'воскресенье']
    for _ in useless:
        text = text.replace(_, '')
    return text

def clean_text(text):
    text = remove_TASS(text)
    text = remove_URL(text)
    text = remove_HTML(text)
    text = remove_numbers(text)
    text = remove_punct(text)
#    text = remove_spaces(text)
    return text

# функция лемматизации
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords \
              and token != " " \
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text

In [23]:
df1 = pd.read_csv("C:/Users/hp/TASS_news/100/ekonomika_100.csv",encoding='utf8',index_col=0)
df2 = pd.read_csv("C:/Users/hp/TASS_news/100/kultura_100.csv",encoding='utf8',index_col=0)
df3 = pd.read_csv("C:/Users/hp/TASS_news/100/nauka_100.csv",encoding='utf8',index_col=0)
df4 = pd.read_csv("C:/Users/hp/TASS_news/100/politika_100.csv",encoding='utf8',index_col=0)
df5 = pd.read_csv('C:/Users/hp/TASS_news/100/sport_100.csv',encoding='utf8',index_col=0)

In [25]:
df_tass = pd.concat([df1,df2,df3,df4,df5], axis = 0,ignore_index=True)
#реиндексация
df_tass = df_tass.reindex(index=[i for i in range(df_tass.shape[0])])
df_tass.to_csv('textsTASS.csv')
print(df_tass.shape[0])

505


In [69]:
df_tass.tail()

,Unnamed: 0,Head,Lead,Text,Rubric,Classi
500,500,"УНИКС победил ""Андорру"" в первом туре топ-16 б...",Встреча завершилась со счетом 73:66,"ТАСС, 13 января. Российский баскетбольный клуб...",sport,2
501,501,"""Зенит"" уступил ""Баварии"" и прервал серию из т...",Встреча закончилась со счетом 80:82,"ТАСС, 13 января. Российский ""Зенит"" на выезде ...",sport,2
502,502,Кущенко: Единая лига ВТБ вынуждена оптимизиров...,"Глава лиги отметил, что в первый день было про...","МОСКВА, 13 января. /Корр. ТАСС Андрей Карташов...",sport,2
503,503,"Шибалов: организаторы на старте этапа ""Дакара""...",Все экипажи стартовали друг за другом с тридца...,"МОСКВА, 13 января. /ТАСС/. Организаторы ралли-...",sport,2
504,504,"Карякин: нас ждут в штабе ""Дакара"" из-за видео...",На девятом этапе ралли-марафона багги российск...,"МОСКВА, 13 января. /ТАСС/. Организаторы ралли-...",sport,2


In [66]:
df_tass = pd.read_csv("C:/Users/hp/textsTASS.csv")

In [68]:
df_tass['Classi'] = df_tass['Rubric'].map({'nauka': 0, 'kultura': 1, 'sport': 2, 'politika': 3, 'ekonomika': 4})

In [72]:
df_tass["Clean_text"] = df_tass["Text"].apply(clean_text)
df_tass["Clean_text"] = df_tass["Clean_text"].apply(preprocess_text)
df_tass["Clean_text"] = df_tass["Clean_text"].apply(remove_months)
df_tass["Clean_text"] = df_tass["Clean_text"].apply(remove_wordsNumbers)
df_tass["Clean_text"] = df_tass["Clean_text"].apply(remove_useless)
df_tass["Clean_text"] = df_tass["Clean_text"].apply(remove_spaces)

In [93]:
df_tass.loc[307]['Clean_text']

'москва обвинение журналист владимир соловьев сторона глава мид латвия эдгарс ринкевичса якобы иметь место эфир героизация нацизм являться чушь заявлять брифинг официальный представитель мид рф мария захаров комментировать запрет соловьев въезд латвия напоминать министр иностранный дело латвия обвинять соловьев прославление нацизм отмечать повидеть ринкевичса возможность посмотреть исходный материал комментировать изза блокировка страна телеканал вгтрк придумывать чушь иначе глава мид латвия знать настолько ярый антифашист владимир соловьев посвящать тема борьба коричневый чума значительный часть свой эфирный время находить просто сложно указывать захаров дипломат подчеркивать данный ситуация цинизм латвийский власть превосходить весь допустимый граница поведение рига требовать квалификация скоро правовой указывать взяться борьба нацизм пора переставать попустительствовать прославлять гражданин латвия добавлять дипломат официальный представитель мид рф также напоминать латвия вместе ря

In [73]:
# векторизация

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
train_tfidf_vec = vec.fit(df_tass['Clean_text'])  
# test_tfidf_vec = vec.fit(df_test_clean['text'])

vec_train = train_tfidf_vec.transform(df_tass['Clean_text'])
# vec_test = test_tfidf_vec.transform(df_test_clean['text'])

In [74]:
print("Shape of the train embedded vector is {}".format(vec_train.shape))

Shape of the train embedded vector is (505, 14769)


In [75]:
x = vec_train
y = df_tass['Classi']

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [77]:
# наивный Байес
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

mnb = MultinomialNB(alpha=1).fit(X_train, y_train)
predicted_mnb = mnb.predict(X_test)

In [78]:
predicted_mnb

array([4, 4, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 0, 0, 3, 0, 2, 1, 1, 0, 0, 3,
       2, 1, 4, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 3, 2, 0, 1, 3, 4, 2, 0,
       4, 4, 2, 3, 2, 1, 3, 1, 0, 1, 1, 4, 3, 1, 3, 0, 4, 0, 3, 2, 3, 0,
       4, 2, 2, 3, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 1,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 4, 2, 1], dtype=int64)

In [79]:
# деревья решений
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=107, min_samples_split=5, min_samples_leaf=1, max_depth=None)

rfc.fit(X_train, y_train)
predicted_rfc = rfc.predict(X_test)

In [47]:
predicted_rfc

array([4, 0, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 0, 0, 3, 0, 2, 1, 1, 0, 0, 3,
       2, 1, 3, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 3, 2, 0, 1, 3, 4, 2, 0,
       4, 4, 2, 3, 2, 1, 3, 1, 0, 1, 1, 4, 4, 1, 3, 0, 4, 0, 3, 2, 3, 0,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 1,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 4, 2, 1], dtype=int64)

In [80]:
predicted_rfc

array([4, 4, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0, 3,
       2, 1, 4, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 4, 2, 0, 1, 3, 4, 2, 0,
       4, 4, 2, 3, 2, 1, 3, 1, 0, 1, 1, 4, 4, 1, 3, 0, 3, 0, 3, 2, 3, 0,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 1,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 3, 2, 1], dtype=int64)

In [81]:
# метод К-ближайших соседей
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
predicted_knn = knn.predict(X_test)

In [49]:
predicted_knn

array([4, 4, 2, 3, 2, 4, 3, 3, 1, 1, 2, 2, 0, 2, 1, 2, 2, 2, 1, 2, 4, 2,
       2, 1, 2, 3, 4, 3, 2, 2, 3, 2, 1, 2, 0, 3, 2, 2, 2, 1, 2, 2, 2, 4,
       2, 4, 2, 3, 2, 1, 2, 1, 0, 2, 2, 4, 2, 1, 3, 0, 4, 0, 2, 2, 2, 1,
       4, 2, 1, 4, 1, 4, 3, 3, 1, 2, 2, 4, 2, 2, 2, 4, 2, 2, 3, 3, 2, 1,
       3, 4, 3, 2, 2, 2, 1, 2, 2, 2, 3, 2, 1], dtype=int64)

In [82]:
predicted_knn

array([4, 4, 2, 3, 2, 4, 2, 3, 1, 1, 2, 2, 0, 2, 1, 2, 2, 2, 1, 2, 4, 2,
       2, 2, 2, 3, 4, 3, 2, 2, 3, 2, 1, 2, 0, 3, 2, 2, 2, 2, 2, 2, 2, 4,
       2, 4, 2, 3, 2, 1, 2, 1, 0, 2, 2, 4, 2, 1, 3, 0, 4, 0, 2, 2, 2, 1,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 2, 2, 4, 2, 2, 2, 4, 2, 2, 3, 3, 2, 1,
       3, 4, 3, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2], dtype=int64)

In [87]:
# метод опорных векторов
from sklearn.svm import SVC
svc = SVC(C=5)
svc.fit(X_train, y_train)
predicted_svc = svc.predict(X_test)

In [51]:
predicted_svc

array([4, 0, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 0, 0, 3, 0, 2, 1, 1, 4, 0, 3,
       2, 1, 4, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 3, 2, 1, 1, 3, 3, 2, 0,
       4, 4, 2, 3, 2, 1, 3, 1, 0, 1, 1, 4, 3, 1, 3, 0, 4, 0, 3, 2, 3, 0,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 0,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 4, 2, 1], dtype=int64)

In [88]:
predicted_svc

array([4, 0, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 0, 0, 1, 0, 2, 1, 1, 4, 0, 3,
       2, 1, 4, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 3, 2, 0, 1, 3, 3, 2, 0,
       4, 4, 2, 3, 2, 1, 3, 1, 0, 1, 1, 4, 3, 1, 3, 0, 4, 0, 3, 2, 3, 0,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 0,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 3, 2, 1], dtype=int64)

In [89]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier (learning_rate=0.02, n_estimators=600, subsample=0.6, min_child_weight=1, max_depth=40, gamma=1, colsample_bytree=1.0)
xgb_model.fit(X_train, y_train)
predicted_xgb = xgb_model.predict(X_test)

C:\Users\hp\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:37:39] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [54]:
predicted_xgb

array([4, 4, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 0, 0, 1, 0, 3, 1, 1, 0, 0, 3,
       2, 1, 4, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 4, 2, 0, 1, 3, 3, 2, 0,
       4, 4, 2, 3, 2, 1, 3, 1, 0, 3, 1, 4, 4, 1, 3, 0, 4, 0, 3, 2, 3, 0,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 1,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 3, 2, 1], dtype=int64)

In [90]:
predicted_xgb

array([4, 4, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 3, 0, 1, 0, 3, 1, 1, 0, 0, 3,
       2, 1, 3, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 4, 2, 0, 1, 3, 4, 2, 0,
       4, 3, 2, 3, 2, 1, 2, 1, 0, 1, 1, 4, 4, 1, 3, 0, 3, 0, 3, 2, 3, 0,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 1,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 3, 2, 1], dtype=int64)

In [91]:
# стекинг моделей
from sklearn.ensemble import StackingClassifier

estimators = [
    ("mnb", mnb),
    ("rfc", rfc),
    ("knn", knn),
    ("svc", svc),
    ("xgb", xgb_model)
]

sc = StackingClassifier(estimators=estimators,cv=5)

sc.fit(X_train, y_train)
predicted_sc = sc.predict(X_test)

C:\Users\hp\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:38:32] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\hp\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:38:52] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\hp\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\hp\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:39:47] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\hp\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:40:07] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [55]:
predicted_sc

array([4, 4, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 0, 0, 3, 0, 2, 1, 1, 0, 0, 3,
       2, 1, 4, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 3, 2, 0, 1, 3, 4, 2, 0,
       4, 4, 2, 3, 2, 1, 3, 1, 0, 1, 1, 4, 3, 1, 3, 0, 4, 0, 3, 2, 3, 0,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 1,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 4, 2, 1], dtype=int64)

In [92]:
predicted_sc

array([4, 4, 2, 3, 3, 4, 3, 3, 1, 1, 2, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0, 3,
       2, 1, 4, 3, 4, 3, 4, 0, 3, 3, 1, 2, 0, 3, 3, 2, 0, 1, 3, 4, 2, 0,
       4, 4, 2, 3, 2, 1, 3, 1, 0, 1, 1, 4, 3, 1, 3, 0, 4, 0, 3, 2, 3, 0,
       4, 2, 2, 4, 1, 4, 3, 3, 1, 0, 2, 4, 2, 0, 0, 0, 0, 3, 3, 3, 2, 1,
       3, 4, 3, 2, 3, 2, 1, 2, 4, 2, 4, 2, 1], dtype=int64)